In [1]:
import pandas as pd 

In [3]:
## todo:
# 1. see how well you can identify the different annotation categories in sentences using simple methods
# 2. see if there's a way to do across-document inference

In [4]:
ls ../data/

annotated-source-training-df.csv
articles3.csv
fasttext-data/
news-article-flatlist/
news-categories-training-df.csv
nyt_corpus/
nytimes-articles-to-extract-sources.csv
output/


# Try to classify Sentence Type

In [7]:
annotated_df = pd.read_csv('../data/annotated-source-training-df.csv', index_col=0)

In [50]:
annotated_df['quote_type'].fillna('None').value_counts()

None                            3056
QUOTE                           1962
BACKGROUND                       572
STATEMENT                        162
PUBLISHED WORK                   134
PRESS REPORT                      92
TWEET                             54
PUBLIC SPEECH, NOT TO JOURNO      53
COMMUNICATION, NOT TO JOURNO      44
PROPOSAL                          32
DOCUMENT                          28
DECLINED COMMENT                  25
Other: LAWSUIT                    17
PROPOSAL/ORDER/LAW                16
Other: Evaluation                  9
LAWSUIT                            8
Other: Campaign filing             7
Other: VOTE/POLL                   6
Other: PROPOSAL                    5
Other: Campaign Filing             4
VOTE/POLL                          3
Other: Analysis                    2
Other: Data analysis               2
Other: Database                    1
Other: LAW                         1
Other: Investigation               1
Other: Data Analysis               1
N

In [51]:
annotated_df[['sent', 'quote_type']]

,sent,quote_type
0,BANGKOK,NaN
1,—,NaN
2,A plane carrying key senior Laotian government...,QUOTE
3,Killed in the crash were two top figures in th...,NaN
4,For a Communist party that relies on force and...,NaN
...,...,...
6292,Scientists had expected an active hurricane se...,NaN
6293,"That expectation has now been exceeded , along...",NaN
6294,After exhausting the 21-name list that was est...,NaN
6295,Mr. Feltgen said that “ 2020 had all the right...,QUOTE


# Try to classify Source Type

In [12]:
annotated_df['source_type'] = annotated_df['source_type'].str.replace('Other: ', '').fillna('None')# .value_counts()

In [15]:
annotated_df['source_type'].value_counts()

None                  5403
Named Individual       520
Unnamed Group          144
Named Group            116
Report/Document         75
Unnamed Individual      32
Vote/Poll                4
Database                 3
Name: source_type, dtype: int64

In [18]:
annotated_df[['sent', 'source_type']].loc[lambda df: df['source_type'] == 'Unnamed Individual']['sent'].iloc[0]

'An agency spokesman said in an email that all nursing homes and assisted living facilities have generators on site .  '

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split

In [32]:
cv = CountVectorizer(min_df=.005, max_df=.5)

In [33]:
lr = LogisticRegressionCV(max_iter=500)

In [ ]:
source_training_df

In [39]:
annotated_df['source_type'].value_counts()

None                  5403
Named Individual       520
Unnamed Group          144
Named Group            116
Report/Document         75
Unnamed Individual      32
Vote/Poll                4
Database                 3
Name: source_type, dtype: int64

In [67]:
(annotated_df
 .loc[lambda df: ~df['source_type'].isin(['Named Individual', 'Named Group'])]
 .dropna()
 ['head'].str.lower().str.strip()
 .value_counts()
 .index.tolist()
)

['officials',
 'experts',
 'analysts',
 'scientists',
 'authorities',
 'economists',
 'report',
 'critics',
 'investigators',
 'prosecutors',
 'woman',
 'government scientists',
 'officers',
 'person',
 'white house officials',
 'people',
 'spokeswoman',
 'associated press',
 'study',
 'forecasters',
 'witnesses',
 'intergovernmental panel on climate change',
 'administration officials',
 'reuters',
 'local media',
 'state television',
 'the cincinnati enquirer',
 'police',
 'environmentalists',
 'administration official',
 'opponents',
 'oil experts',
 'new york times investigation',
 'the anadolu news agency',
 'proponents-2',
 'spokeswoman-1',
 'opponents-2',
 'mississippi official',
 'auditors',
 'news reports',
 'speculation',
 'every list',
 'united states federal aviation administration',
 'kathleen sgamma',
 'surrogates',
 'the soma mining company',
 'lauren bonner',
 'unnamed',
 'city officials',
 'one man',
 'another man',
 'several people',
 'pilot',
 'four people',
 'the ch

In [34]:
X_train, X_test, y_train, y_test = train_test_split(annotated_df['sent'], annotated_df['source_type'])

In [40]:
X_mat_train = cv.fit_transform(X_train)

In [41]:
lr.fit(X_mat, y_train)

/Users/alex/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/Users/alex/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/alex/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iter

LogisticRegressionCV(max_iter=500)

In [42]:
X_mat_test = cv.transform(X_test)

In [45]:
pd.Series(lr.predict(X_mat_test)).value_counts()

None                1523
Named Individual      51
Unnamed Group          1
dtype: int64